In [23]:
import torch
from torch import nn
from model import Encoder, AttentionDecoder, EMBEDDING_SIZE, SOS_token
import random
from prepared import load_voc, batch2train_data

In [24]:
data, Vocabulary = load_voc()

## Подготовка данных, для использования в модели


In [25]:
inp, lenghts, target, mask, max_target_len = batch2train_data([random.choice(data) for _ in range(2)])

In [26]:
inp.size()

torch.Size([7, 2])

In [27]:
target.size()

torch.Size([27, 2])

In [28]:
Vocabulary.num_words

35691

In [29]:
embedding = nn.Embedding(2**16, EMBEDDING_SIZE)
encoder = Encoder(embedding)
decoder = AttentionDecoder(embedding)

In [30]:
device = "cuda:0"

In [31]:
embedding.num_embeddings

65536

In [32]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()


In [33]:
decoder_input = torch.ones(64).view((1, 64)).long()

In [34]:
training_batches = [batch2train_data([random.choice(data) for _ in range(64)]) for _ in range(128)]

In [35]:
inp, lenghts, out, mask, max_target_lenght = training_batches[0]

In [36]:
decoder_input.shape

torch.Size([1, 64])

In [37]:
random.choice(data)

['ну так скажите  куда мы  по вашему  едем ',
 'куда  куда  тебе что  услышать хочется лишний раз  ясное дело  куда  к разрушенному мосту  что ты себе смолоду такой херней голову забиваешь  андрюха ']

In [38]:
encoder_out, encoder_hidden = encoder(inp, lenghts)

In [39]:
embedding(inp)

tensor([[[ 0.3761, -0.9388, -0.7662,  ..., -0.7292,  0.2528, -1.5036],
         [ 0.2781,  2.2181, -1.8477,  ..., -2.2462,  0.6356,  0.6831],
         [-0.9759,  0.1534, -0.1476,  ...,  0.4621, -0.5021,  1.0112],
         ...,
         [ 0.6782, -0.0552, -0.0959,  ...,  1.1213, -1.1809,  0.8640],
         [-0.0207,  1.5888, -0.3059,  ..., -0.2259,  0.9216,  0.5446],
         [-1.6445,  0.5777, -1.7869,  ...,  0.1063,  0.5181, -0.1051]],

        [[ 0.4501, -0.1442, -1.1952,  ...,  1.9488,  0.8456,  0.3391],
         [-0.6323,  1.2789,  1.0246,  ...,  0.3898,  0.4185,  0.7187],
         [-1.5673, -1.2672,  0.1128,  ..., -0.6350,  0.6364,  0.5654],
         ...,
         [ 0.3034, -0.3745,  0.0764,  ...,  0.4210,  0.0210,  1.2462],
         [ 1.6393,  0.1444,  1.2381,  ...,  0.1740, -0.6929, -0.0708],
         [ 1.6393,  0.1444,  1.2381,  ...,  0.1740, -0.6929, -0.0708]],

        [[-0.0659,  0.6104, -1.6969,  ..., -0.0707,  0.9871,  1.8567],
         [ 0.6835, -1.0751, -0.2272,  ..., -1

In [40]:
inp

tensor([[   80,     9,   710,  ...,  1161,   582, 30543],
        [  127,   272, 12638,  ...,     4,     2,     2],
        [   39,  1126, 15353,  ...,     2,     0,     0],
        ...,
        [    4,     4,     0,  ...,     0,     0,     0],
        [    4,     2,     0,  ...,     0,     0,     0],
        [    2,     0,     0,  ...,     0,     0,     0]])

In [41]:
encoder_hidden[:2].shape

torch.Size([2, 64, 128])

In [42]:
encoder_hidden.shape

torch.Size([4, 64, 128])

In [43]:
decoder(decoder_input, encoder_hidden[:2], encoder_out)

(tensor([[1.4556e-05, 1.5725e-05, 1.4634e-05,  ..., 1.5478e-05, 1.4346e-05,
          1.5756e-05],
         [1.3723e-05, 1.6123e-05, 1.5011e-05,  ..., 1.6184e-05, 1.4667e-05,
          1.6344e-05],
         [1.4097e-05, 1.6384e-05, 1.5196e-05,  ..., 1.4237e-05, 1.4292e-05,
          1.5084e-05],
         ...,
         [1.3370e-05, 1.6301e-05, 1.5298e-05,  ..., 1.6102e-05, 1.3937e-05,
          1.5765e-05],
         [1.3563e-05, 1.5556e-05, 1.6594e-05,  ..., 1.4885e-05, 1.4375e-05,
          1.5212e-05],
         [1.4848e-05, 1.6134e-05, 1.7080e-05,  ..., 1.6174e-05, 1.4911e-05,
          1.4783e-05]], grad_fn=<SoftmaxBackward>),
 tensor([[[ 1.8600e-01, -1.7920e-01, -2.0808e-01,  ..., -5.7619e-01,
            4.3289e-01,  3.3865e-01],
          [ 3.8053e-01, -4.2061e-02, -1.6946e-01,  ..., -6.6075e-01,
            5.8597e-01,  7.0384e-03],
          [-2.8307e-02,  4.2333e-01, -4.0583e-02,  ..., -3.7236e-01,
            2.6082e-01,  3.1713e-01],
          ...,
          [ 8.5236e-02,  1.

In [51]:
embed = embedding(decoder_input)

In [52]:
embed, hidden = decoder.gru(embed)

In [53]:
embed

tensor([[[-0.0258, -0.1106, -0.0412,  ...,  0.0154,  0.0369,  0.1458],
         [-0.0294, -0.1166,  0.0214,  ..., -0.0258,  0.0208,  0.1486],
         [-0.0536,  0.0036, -0.0253,  ...,  0.0428,  0.0175,  0.0770],
         ...,
         [-0.0633, -0.0350, -0.0175,  ..., -0.0289,  0.1048,  0.1651],
         [-0.0267,  0.0866, -0.0254,  ..., -0.0310, -0.0892,  0.1229],
         [ 0.0429, -0.0371, -0.0593,  ..., -0.0896, -0.0579,  0.1566]]],
       grad_fn=<StackBackward>)

In [48]:
hidden * 